In [1]:
!pip install torch

    100% |████████████████████████████████| 676.9MB 40kB/s  eta 0:00:01                | 190.6MB 39.8MB/s eta 0:00:13    63% |████████████████████▎           | 428.0MB 38.9MB/s eta 0:00:07    67% |█████████████████████▋          | 458.0MB 27.7MB/s eta 0:00:08    94% |██████████████████████████████  | 636.3MB 39.4MB/s eta 0:00:02


In [2]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint

import numpy as np
import torch
from numpy import dot, absolute
from numpy.linalg import norm

## Load model

In [3]:
# Load model
from models import InferSent
model_version = 1
MODEL_PATH = "../encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [4]:
# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model

In [5]:
# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [6]:
import pandas as pd
import ast
#folder_prefix = '/home/jovyan/work/'
df = pd.read_csv('limited_pages_text.csv')

In [7]:
!pip install nltk
!pip install tqdm
!pip install gensim

    100% |████████████████████████████████| 3.1MB 8.1MB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/97/8a/10/d646015f33c525688e91986c4544c68019b19a473cb33d3b55
Successfully built nltk
    100% |████████████████████████████████| 51kB 3.8MB/s ta 0:00:011
    100% |████████████████████████████████| 24.2MB 1.7MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 26.9MB/s ta 0:00:01
    100% |████████████████████████████████| 1.4MB 14.0MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 38.7MB/s ta 0:00:01
    100% |████████████████████████████████| 5.5MB 6.7MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 15.4MB/s ta 0:00:01
    100% |████████████████████████████████| 552kB 33.3MB/s ta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/b8/cb/43/c0ba52baf2b0e371ec1d5b2d4685d6d24617b1391f3eeacda5
Successfully built smart-open


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
df_new = df[df.text_full.isnull() == False]

In [10]:
df_new = df_new.reset_index(drop=True)

In [11]:
df_new['word count'] = df_new.text_full.apply(lambda x: len(x.split()))

In [12]:
df_new[:800]['word count'].max()

24010

In [13]:
df_new[800:1600]['word count'].max()

96639

In [14]:
df_new[1600:2400]['word count'].max()

23604

In [15]:
df_new[2400:3200]['word count'].max()

32520

In [16]:
df_new[3200:4000]['word count'].max()

81042

In [17]:
df_new[4000:4800]['word count'].max()

22444

In [18]:
df_new[4800:5600]['word count'].max()

66574

In [19]:
df_new[5600:]['word count'].max()

33338

RNN can run into memory trouble if there are far too many words inputted. Therefore, we will limit our words to 20000 max and cut off any excess words after that per row.

In [20]:
df_new['text_full_limited'] = df_new['text_full'].apply(lambda x: ' '.join(x.split()[:10000]) 
                                                        if len(x.split()) > 10000 else x)

In [21]:
df_new['word count new'] = df_new.text_full_limited.apply(lambda x: len(x.split()))

In [22]:
# Load embeddings of K most frequent words
#model.build_vocab_k_words(K=100000)
#once we have the sentences ready
model.build_vocab(df_new['text_full_limited'])

Found 85508(/184194) words with w2v vectors
Vocab size : 85508


## Encode sentences

In [12]:
# gpu mode : >> 1000 sentences/s
# cpu mode : ~100 sentences/s

In [23]:
#splitting the data into 8 as full data can't be run with our ram in one go
text_full_1 = df_new['text_full_limited'][:800]
text_full_2 = df_new['text_full_limited'][800:1600]
text_full_3 = df_new['text_full_limited'][1600:2400]
text_full_4 = df_new['text_full_limited'][2400:3200]
text_full_5 = df_new['text_full_limited'][3200:4000]
text_full_6 = df_new['text_full_limited'][4000:4800]
text_full_7 = df_new['text_full_limited'][4800:5600]
text_full_8 = df_new['text_full_limited'][5600:]

In [24]:
#text_full_1 = df_new['text_full'][:800]
embeddings1 = model.encode(text_full_1, verbose=True)
print('schools encoded : {0}'.format(len(embeddings1)))

Nb words kept : 1830611/1878251 (97.5%)
Speed : 0.2 sentences/s (cpu mode, bsize=64)
schools encoded : 800


In [ ]:
embeddings2 = model.encode(text_full_2, verbose=True)
print('# schools encoded : {0}'.format(len(embeddings2)))

Nb words kept : 1973049/2028362 (97.3%)


In [ ]:
embeddings3 = model.encode(text_full_3, verbose=True)
print('# schools encoded : {0}'.format(len(embeddings3)))

In [ ]:
embeddings4 = model.encode(text_full_4, verbose=True)
print('# schools encoded : {0}'.format(len(embeddings4)))

In [ ]:
embeddings5 = model.encode(text_full_5, verbose=True)
print('# schools encoded : {0}'.format(len(embeddings5)))

In [ ]:
embeddings6 = model.encode(text_full_6, verbose=True)
print('# schools encoded : {0}'.format(len(embeddings6)))

In [ ]:
embeddings7 = model.encode(text_full_7, verbose=True)
print('# schools encoded : {0}'.format(len(embeddings7)))

In [ ]:
embeddings8 = model.encode(text_full_8, verbose=True)
print('# schools encoded : {0}'.format(len(embeddings8)))

Running into CPU memory allocation error. As a result, we'll attempt to find the cosine similarities of the first 800 schools with the terminologies.

Much of the code below referenced from correlation_plot.ipynb 

In [ ]:
#writing function for common cosine similarity
def doc_words_cosine(ncessch, nterms):
    i = df_new[df_new.NCESSCH == ncessch].index[0]
    emb = embeddings2[i] #change this to embeddings2, 3, etc. Due to memory have to do it chunk by chunk
    if nterms == 5: #seed
        word_vec_avg = np.sum(inqseed_embeddings, axis=0)/5
    elif nterms == 30:
        word_vec_avg = np.sum(inq30_embeddings, axis=0)/30
    elif nterms == 100:
        word_vec_avg = np.sum(inq100_embeddings, axis=0)/100
    elif nterms == 500:
        word_vec_avg = np.sum(inq500_embeddings, axis=0)/100
    return absolute(dot(emb, word_vec_avg)/(norm(emb)*norm(word_vec_avg)))

In [26]:
inqseed = ['inquiry-based', 'problem-based', 'discovery', 'experiential', 'constructivist']

inq30 = ['discovery-based', 'student-driven_exploration', 'exploration_and_experimentation', 'laboratory-based', 
         'problem-based', 'prbl', 'learn-by-doing', 
         'project-based', 'project-centered', 
         'experiential', 'experiential_approach', 'experientially',
         'inquiry-based', 'inquiry-driven', 'student-centered_inquiry-based', 'active_inquiry', 
         'constructivist', 'constructivism', 
         'hands-on', 'hand-on', 'hands-on_learning', 'hands-on_and_minds-on', 'hands-on_minds-on', 'hands-on/minds-on', 
         'socratic', 'socratic_method', 'socratic_dialogue',
         'child-centered', 'learner-centered', 'student-centered']

inq100 = ['discovery-based', 'student-driven_exploration', 'exploration_and_experimentation', 'laboratory-based', 'problem-based', 'prbl', 'learn-by-doing', 'project-based', 'project-centered', 'experiential', 'experiential_approach', 'experientially', 'inquiry-based', 'inquiry-driven', 'student-centered_inquiry-based', 'active_inquiry', 'constructivist', 'constructivism', 'hands-on', 'hand-on', 'hands-on_learning', 'hands-on_and_minds-on', 'hands-on_minds-on', 'hands-on/minds-on', 'socratic', 'socratic_method', 'socratic_dialogue', 'child-centered', 'learner-centered', 'student-centered', 'arithmetic_reasoning', 'climate_improvement/improvement', 'awaken_the_imagination', 'cross-content', 'abilities_will_be_maximized', 'core-content_courses', 'benefits_of_co-op', 'proxy_servers', 'twinery', 'theme-based', 'relevant_to_the_viability', 'interdisciplinary', 'cogent', 'open-mindedness', 'reinforce_previously', 'equipping_educators', 'mentorships', 'nonpolitical', 'trained_appraisers', 'lightness', 'respective_histories', 'instruction_cgi', 'sustaining_of_a_genuine', 'natural', 'case-based', 'indispensable', 'timeless_but_indispensable', 'prohibitively_expensive', 'disparate_personalities', 'inspiring', 'richness_and_strength', 'experience-based', 'thorough_and_accurate', 'vitals', 'obsolescence', 'ellipsis_points', 'circles_graphed', 'polite_manners', 'homeschool-friendly', 'participating_in_acting/music/arts', 'develop/implement', 'willow_seedling', 'non-coercive', 'gender-nonconforming', 'activity/project', 'identification_of_learning/behavioral', 'orientating', 'interdependence', 'individual/personalized_manner', 'technological_modalities', 'experimentation', 'particularly_vulnerable', 'curiosity', 'discerning_listeners', 'approaches_such_as_sheltered', 'rule-making', 'dissipate', 'mirror_real-life', 'product/outcome', 'expeditions', 'wishful', 'conceiving', 'independence', 'drove_the_expedition', 'thrives', 'functional_weakness', 'would_be_off-putting', 'collaborative_student-community', 'trustee_elections', 'evaluate_the_motives']

inq500 = ['active_inquiry',
 'child-centered',
 'constructivism',
 'constructivist',
 'discovery-based',
 'experiential',
 'experiential_approach',
 'experientially',
 'exploration_and_experimentation',
 'hand-on',
 'hands-on',
 'hands-on/minds-on',
 'hands-on_and_minds-on',
 'hands-on_learning',
 'hands-on_minds-on',
 'inquiry-based',
 'inquiry-driven',
 'laboratory-based',
 'learn-by-doing',
 'learner-centered',
 'prbl',
 'problem-based',
 'project-based',
 'project-centered',
 'socratic',
 'socratic_dialogue',
 'socratic_method',
 'student-centered',
 'student-centered_inquiry-based',
 'student-driven_exploration',
 'abilities',
 'abilities_will_be_maximized',
 'ability-grouped',
 'acquire_requisite',
 'activities',
 'activities/lessons',
 'activity/project',
 'adept_at_facilitating',
 'adherents',
 'adjusted_and_thrived',
 'admittedly_fallible',
 'agile',
 'aligned_to_the_ngss',
 'allegiances',
 'altered/added',
 'amazed_by_the_generosity',
 'anecdotes',
 'appealing_and_relevant',
 'approaches_such_as_sheltered',
 'appropriately_allocated',
 'architecture',
 'arguments_and_informative/explanatory',
 'arithmetic_reasoning',
 'art-filled',
 'artistic',
 'assemble_the_pieces',
 'atomic_nucleus',
 'awaken_the_imagination',
 'bearing_on_the_viability',
 'benefits_of_co-op',
 'bi-literacy',
 'brain-compatible',
 'brain-friendly',
 'break/gross_motor',
 'brighter_tomorrow',
 'brilliantly_infused',
 'brochure_to_solidify',
 'c++_programming',
 'capitalism_and_socialism',
 'career-focused_and_technical',
 'carefully_weigh',
 'case-based',
 'championing_issues',
 'channels_their_energies',
 'character-based_strengths',
 'checking-for-understanding',
 'child-centered_individualized',
 'child-sized_furniture',
 'circles_graphed',
 'circulatory_and_nervous',
 'city-planning',
 'civically_active',
 'classical/stem_based',
 'climate_improvement/improvement',
 'coaches/leaders',
 'cogent',
 'cognitively_demanding',
 'coherent_sequenced',
 'cohesiveness',
 'collaborate_on_a_large-scale',
 'collaborative_student-community',
 'college-level_options',
 'colorism',
 'comma_rules',
 'commensurate_with_their_abilities',
 'committing_offenses',
 'commonly_confused',
 'communication/collaboration',
 'community-wide',
 'comparing_the_inputs',
 'compelling_paragraphs',
 'competency_and_mastery',
 'complex_adult-world',
 'complexity',
 'comprehensibility',
 'comprehensive_16-hour',
 'conceiving',
 'concept/skill',
 'concurrently_with_the_choreography',
 'considering_waldorf',
 'constructivist-based',
 'consultation/collaboration',
 'content-rich_nonfiction',
 'content-specific',
 'context',
 'continous',
 'continuity_and_commitment',
 'core-content_courses',
 'craftsmanship',
 'create_spectacles',
 'creative',
 'creativity',
 'critical-thinkers',
 'critical-thinking',
 'cross-content',
 'crystallization',
 'cultural_diversities',
 'cultural_heritages',
 'curiosity',
 'curriculum/standards',
 'data-rich',
 'decision-making',
 'decisionmaking_skills',
 'deepen',
 'deeper',
 'definiteness',
 'demanding_college-preparatory',
 'derivations',
 'describe_distributions',
 'design2fab_lab',
 'develop/implement',
 'developing',
 'dialoguing',
 'diaries_and_journals',
 'different/',
 'differentiated/personalized',
 'differing_structure',
 'discerning_listeners',
 'discover',
 'disempower',
 'disparate_personalities',
 'dissection_techniques',
 'dissipate',
 'distraction-free',
 'drafting_and_revising',
 'drove_the_expedition',
 'dynamic_collaboration',
 'dynamically_and_immediately',
 'edifying',
 'editing_and_proofing',
 'effectiveness_of_metacognitive',
 'effects_of_repetition',
 'efficacy_by_encouraging',
 'eliminating_unnecessary',
 'elitism',
 'ellipsis_points',
 'embraced_and_supported',
 'emojis',
 'encourage',
 'encouraging',
 'enduring_conceptual',
 'enduring_relationships',
 'engaged_citizen',
 'engaging',
 'enlargement',
 'ennoble',
 'ennobling_effect',
 'environment',
 'equipping_educators',
 'esl/family',
 'evaluate_the_motives',
 'evidence_in_a_nuanced',
 'examples_evoke',
 'expeditions',
 'experience-based',
 'experiental_learning',
 'experimental_labs',
 'experimentation',
 'exploration',
 'explorations',
 'explorative',
 'expose_them_to_a_wider',
 'exposed_to_a_coherent',
 'facilitation_techniques',
 'faint_of_heart',
 'feedback',
 'field-based',
 'fieldwork',
 'filming',
 'fiscal_mismanagement',
 'force_to_be_reckoned',
 'fostered',
 'fostering',
 'fuels_my_desire',
 'fully-vetted',
 'functional_weakness',
 'gender-nonconforming',
 'generalizations_and_predictions',
 'gets_maximized',
 'goal-driven',
 'gods_of_asgard',
 'grade-by-grade_year-by-year',
 'gradual_manner',
 'grander_scale',
 'grapple_with_difficult',
 'greek_philosophers',
 'grit_and_zest',
 'growth',
 'hands‑on',
 'harmonize',
 'headmaster_letters',
 'helps',
 'higher-level_cognitive',
 'highly_scripted',
 'home-away-from-home',
 'homeschool-friendly',
 'hope',
 'humanism',
 'humanities_major',
 'identification_of_learning/behavioral',
 'immersive',
 'implant_into_the_memories',
 'in-demand_transportable',
 'incorporating_the_principles',
 'increased_connectedness',
 'independence',
 'indispensable',
 'individual/personalized_manner',
 'individualize_and_differentiate',
 'inflected',
 'innovativeness',
 'inquiry/research',
 'inspire',
 'inspiring',
 'instruction_cgi',
 'integrated',
 'intelligent_and_intentional',
 'inter-dependence',
 'interdependence',
 'interdisciplinary',
 'interests',
 'interior_monologue',
 'internship/interest',
 'interpersonal',
 'investigate',
 'issues-based',
 'item/category_to_better',
 'jasmine_flower',
 'journaling',
 'joy-filled',
 'kinaesthetic',
 'laborious_process',
 'language-rich',
 'leader_and_a_follower',
 'learn',
 'learning',
 'learning-to-learn',
 'lens',
 'life-based',
 'life-long',
 'life-skill',
 'lifelong',
 'lightness',
 'literature-rich',
 'literature_has_tended',
 'loving_and_attentive',
 'low-ability',
 'lowest_in_achievement',
 'loyalty',
 'math/engineering',
 'maturationally_ready',
 'meaningful',
 'mentorship',
 'mentorships',
 'metacognitive',
 'micro-economic_ventures',
 'minds',
 'minds-on',
 'mirror_real-life',
 'mixed-ability',
 'modality',
 'motivate_their_attempts',
 'mud_hut',
 'multi-dimensional',
 'multi-disciplinary',
 'multi-faceted',
 'multiple-paragraph',
 'must-reads',
 'mutual',
 'myon_reader',
 'mythological_creature',
 'natural',
 'nature',
 'nature-based_perspective',
 'navigable',
 'near-daily_basis',
 'need-to-know_facts',
 'needing_improvement',
 'neuro-anatomy',
 'next_generation',
 'non-arts',
 'non-coercive',
 'non-dominant',
 'non-instrumental',
 'non-montessori',
 'noninfectious',
 'nonpolitical',
 'novel-based',
 'nurtures_mutual',
 'nurturing',
 'observation',
 'obsolescence',
 'one-segment',
 'one2one_digital',
 'open-mindedness',
 'orientating',
 'originated_in_cuba',
 'outcome-focused',
 'pace_most_suited',
 'parabolas_graphed',
 'parent/student/staff_compact',
 'participating_in_acting/music/arts',
 'particularly_vulnerable',
 'pedagogical_delivery',
 'pep-rally',
 'personalized',
 'perspective',
 'philosophical',
 'philosophies',
 'photographing',
 'pinata',
 'point-of-need',
 'polite_manners',
 'post-production_editing',
 'post-school_life',
 'power_of_student-faculty',
 'powers_of_abstraction',
 'practicality',
 'pre-teach',
 'precise_claim',
 'predicated_on_the_realization',
 'presumption_and_belief',
 'problem‑solving',
 'process-based_writing',
 'processes_recursively',
 'product/outcome',
 'prohibitively_expensive',
 'project-focused',
 'providing',
 'proxy_servers',
 'purveyors',
 'qualify_or_justify',
 'quality-learning',
 'quantitatively_as_well',
 'quiet',
 'ramp-up_to_readiness',
 're-forming',
 'real-life',
 'real-world',
 'reason_mathematically',
 'rectangular_array',
 'reduces_distractions',
 'refine_gross',
 'reinforce_previously',
 'reinterpret',
 'reinvention',
 'relatable_characters',
 'relational_capacity',
 'relationship-oriented_classroom',
 'relationships_between_pollinators',
 'relevant_to_the_viability',
 'reliability',
 'repeated_occurrence',
 'research-supported',
 'respective_histories',
 'review_of_progress-monitoring',
 'reﬂective',
 'richness_and_diversity',
 'richness_and_strength',
 'robot_to_maneuver',
 'rule-making',
 'scaled_scores',
 'school-life',
 'sculptural_forms',
 'sculptural_techniques',
 'self-appreciation',
 'self-awareness',
 'self-efficacy',
 'self-expression',
 'self-motivated',
 'sense_of_reverence',
 'sensory-motor',
 'sensory-rich',
 'sentence_workshopping',
 'settled_tribe',
 'shapes_and_their_attributes',
 'shared',
 'sherry_alam',
 'shows_that_metacognitive',
 'simulations_role-playing',
 'skills/family',
 'small-group',
 'small-group_discussion',
 'social/soft',
 'socialresponsibility',
 'standards-focused',
 'staunch_supporter',
 'stepstone_360°',
 'stewards',
 'stopped_and_looked',
 "story's_lesson",
 'stress-management',
 'strict/warm',
 'structurally_similar',
 'structured_and_predictable',
 'struggles_and_questioning',
 'student-directed',
 'student-driven',
 'study/teaching',
 'subject-area_integration',
 'succesful',
 'suit_their_needs',
 'superbowl_party',
 'supplemental_curricula',
 'sustainability',
 'sustaining_of_a_genuine',
 'symbolic_representations',
 'synthetic_phonics',
 'systematic_and_sequential',
 'tacitly',
 'target/objective',
 'teacher-to-teacher',
 'team-based_organizational',
 'techniques_namely',
 'technological_modalities',
 'technologies_and_our_science-based',
 'technology-assisted',
 'technology-infused',
 'technology–',
 'tedxyouth',
 'temper_tantrum',
 'text-analysis',
 'text_capitalcitypcs',
 'textbooks_affording',
 'thematic',
 'thematically',
 'theme-based',
 'themes',
 'theses',
 'thorough_and_accurate',
 'three-part_pattern',
 'thrives',
 'tight-knit',
 'time-management',
 'timeless_but_indispensable',
 'touchpoints',
 'trained_appraisers',
 'transgender',
 'tricycle',
 'trustee_elections',
 'tunnels_and_snowshed',
 'twinery',
 'unbidden_encounter',
 'understand',
 'understands',
 'unfamiliar_contexts',
 'unschooling',
 'uphold_and_enforce',
 'valued',
 'values-centered',
 'virtues',
 'virtuous',
 'visible_men',
 'vitals',
 'voluntarism',
 'wampanoag',
 'warm/strict',
 'ways_of_knowing/learning',
 'well-articulated_standards-based',
 'well-developed',
 'whole-group_explicit',
 'willow_seedling',
 'wishful',
 'world-view',
 'would_be_off-putting',
 'writing-centered',
 'writing—decoding',
 'yet-to-be-defined_future']

In [27]:
inqseed

['inquiry-based',
 'problem-based',
 'discovery',
 'experiential',
 'constructivist']

In [28]:
d = {'terms': inqseed}
inqseed_df = pd.DataFrame(d)

In [29]:
inqseed_embeddings = model.encode(inqseed_df['terms'], verbose=True)

Nb words kept : 15/15 (100.0%)
Speed : 15.1 sentences/s (cpu mode, bsize=64)


In [30]:
d = {'terms': inq30}
inq30_df = pd.DataFrame(d)
inq30_embeddings = model.encode(inq30_df['terms'], verbose=True)

d = {'terms': inq100}
inq100_df = pd.DataFrame(d)
inq100_embeddings = model.encode(inq100_df['terms'], verbose=True)

d = {'terms': inq500}
inq500_df = pd.DataFrame(d)
inq500_embeddings = model.encode(inq500_df['terms'], verbose=True)

Nb words kept : 77/90 (85.6%)
Speed : 83.3 sentences/s (cpu mode, bsize=64)
Nb words kept : 240/300 (80.0%)
Speed : 215.9 sentences/s (cpu mode, bsize=64)
Nb words kept : 1182/1494 (79.1%)
Speed : 225.5 sentences/s (cpu mode, bsize=64)


In [31]:
print("shape of seed embeddings: ", inqseed_embeddings.shape)
print("shape of 30 words' embeddings: ", inq30_embeddings.shape)
print("shape of 100 words' embeddings: ", inq100_embeddings.shape)
print("shape of 500 words' embeddings: ", inq500_embeddings.shape)

shape of seed embeddings:  (5, 4096)
shape of 30 words' embeddings:  (30, 4096)
shape of 100 words' embeddings:  (100, 4096)
shape of 500 words' embeddings:  (498, 4096)


In [ ]:
#fragmenting and saving the dataframe
#starting with first 800 for which we have the sentence embeddings
df2 = df_new[800:1600]
#initializing list for appending cosine values
seed_ls = []
for i in range(len(df2)):
    seed_ls.append(doc_words_cosine(df2.NCESSCH[i], 5))
thirty_ls = []
for i in range(len(df2)):
    thirty_ls.append(doc_words_cosine(df2.NCESSCH[i], 30))
hundred_ls = []
for i in range(len(df2)):
    hundred_ls.append(doc_words_cosine(df2.NCESSCH[i], 100))
fivehundred_ls = []
for i in range(len(df2)):
    fivehundred_ls.append(doc_words_cosine(df2.NCESSCH[i], 500))

In [ ]:
df2['infersent_inqseed_cosine'] = seed_ls
df2['infersent_inq30_cosine'] = thirty_ls
df2['infersent_inq100_cosine'] = hundred_ls
df2['infersent_inq500_cosine'] = fivehundred_ls

In [ ]:
df2.to_csv("cosine_scores_infersent_batch2.csv")

In [39]:
df1

,Unnamed: 0,NCESSCH,text_full,infersent_inqseed_cosine,infersent_inq30_cosine,infersent_inq100_cosine,infersent_inq500_cosine
0,0,1.001970e+10,reading buddy the reading buddy program design...,0.244569,0.155203,0.546823,0.641224
1,2,2.001500e+10,select school select school ketchikan high sch...,0.283642,0.193733,0.567076,0.655399
2,3,2.001500e+10,l enrollment homeschool title ix assuring gend...,0.238978,0.151985,0.539373,0.631230
3,4,2.001800e+10,select school select school abbott loop elemen...,0.247802,0.162715,0.544508,0.634701
4,8,2.001800e+10,eer info wcc wpg browse home wpg wpg welcome w...,0.273282,0.182033,0.562822,0.654254
5,10,2.001800e+10,es login frontier's purpose helping parents he...,0.264640,0.174310,0.556891,0.648450
6,13,2.001800e+10,select school select school abbott loop elemen...,0.238087,0.154064,0.536896,0.627592
7,14,2.001800e+10,select school select school abbott loop elemen...,0.232454,0.146983,0.535410,0.627788
8,15,2.002100e+10,juneau community charter school jccs a learnin...,0.254800,0.167114,0.555752,0.648888
9,17,2.003900e+10,fireweed academy soundview ave homer skip cont...,0.242428,0.149574,0.531230,0.622844
